In [1]:
import pandas as pd
import random
import os
import numpy as np
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
import math
import os
import torch
import random
import pandas as pd
import numpy as np
from sklearn import datasets, ensemble
from catboost import CatBoostRegressor
from tqdm import tqdm
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

from sklearn.model_selection import KFold
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate, train_test_split
import itertools
import math
import warnings
warnings.filterwarnings(action='ignore')

In [11]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')
train

,SAMPLE_ID,ARI_CO,ARI_PO,SHIP_TYPE_CATEGORY,DIST,ATA,ID,BREADTH,BUILT,DEADWEIGHT,...,V_WIND,AIR_TEMPERATURE,BN,ATA_LT,DUBAI,BRENT,WTI,BDI_ADJ,PORT_SIZE,CI_HOUR
0,TRAIN_000000,CN,EKP8,Bulk,30.736578,2020-10-15 4:03,Z517571,30.0,28,73100,...,3.77,15.9,2.730798,12,42.01,43.16,40.96,1407.668330,0.001660,3.048333
1,TRAIN_000001,CN,EUC8,Container,63.220425,2019-09-17 2:55,U467618,30.0,15,37900,...,-6.72,24.5,4.289058,10,67.53,64.55,59.34,2089.046774,0.001614,17.138611
2,TRAIN_000002,CN,NGG6,Container,90.427421,2019-02-23 6:43,V378315,50.0,7,115000,...,0.00,9.4,0.000000,14,65.30,66.39,56.94,603.193047,0.001743,98.827500
3,TRAIN_000003,JP,TMR7,Cargo,0.000000,2020-09-18 22:06,B726632,10.0,33,1490,...,-7.31,22.1,4.693735,7,43.02,43.15,41.11,1169.853455,0.000069,0.000000
4,TRAIN_000004,RU,NNC2,Container,8.813725,2022-08-13 12:57,D215135,30.0,10,27600,...,2.31,22.8,2.345875,14,90.45,93.65,88.11,1107.944894,0.000197,96.030556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367436,TRAIN_367436,CN,YRT6,Bulk,59.018184,2017-11-11 22:23,J661243,40.0,13,93200,...,NaN,NaN,NaN,6,61.25,62.21,55.70,1333.609109,0.000360,65.850000
367437,TRAIN_367437,JP,QYY1,Tanker,0.000000,2022-04-29 2:58,D847216,10.0,9,1280,...,0.87,17.1,1.028558,11,105.37,109.34,104.69,1955.103846,0.000552,0.000000
367438,TRAIN_367438,SG,GIW5,Container,1.768630,2022-07-14 7:58,Q635545,30.0,6,25000,...,3.36,31.7,2.557156,15,97.73,99.10,95.78,1601.291086,0.002615,0.997500
367439,TRAIN_367439,JP,TMR7,Cargo,0.000000,2020-12-22 10:07,N211282,10.0,8,2400,...,-2.44,10.8,3.055715,19,49.75,50.08,47.02,1191.353331,0.000069,0.000000


In [12]:
train = train[train['DIST'] != 0]
train = train.reset_index(drop = True)
train

,SAMPLE_ID,ARI_CO,ARI_PO,SHIP_TYPE_CATEGORY,DIST,ATA,ID,BREADTH,BUILT,DEADWEIGHT,...,V_WIND,AIR_TEMPERATURE,BN,ATA_LT,DUBAI,BRENT,WTI,BDI_ADJ,PORT_SIZE,CI_HOUR
0,TRAIN_000000,CN,EKP8,Bulk,30.736578,2020-10-15 4:03,Z517571,30.0,28,73100,...,3.77,15.9,2.730798,12,42.01,43.16,40.96,1407.668330,0.001660,3.048333
1,TRAIN_000001,CN,EUC8,Container,63.220425,2019-09-17 2:55,U467618,30.0,15,37900,...,-6.72,24.5,4.289058,10,67.53,64.55,59.34,2089.046774,0.001614,17.138611
2,TRAIN_000002,CN,NGG6,Container,90.427421,2019-02-23 6:43,V378315,50.0,7,115000,...,0.00,9.4,0.000000,14,65.30,66.39,56.94,603.193047,0.001743,98.827500
3,TRAIN_000004,RU,NNC2,Container,8.813725,2022-08-13 12:57,D215135,30.0,10,27600,...,2.31,22.8,2.345875,14,90.45,93.65,88.11,1107.944894,0.000197,96.030556
4,TRAIN_000005,CN,NGG6,Container,81.435335,2015-09-08 14:24,Z156413,30.0,22,18100,...,NaN,NaN,NaN,22,45.75,48.89,45.92,820.288044,0.001743,42.078056
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220050,TRAIN_367433,IN,UJM2,Bulk,30.199074,2022-03-23 8:35,Y242521,30.0,2,63500,...,3.54,36.5,4.306719,14,111.93,120.65,113.90,2077.159292,0.000217,53.400833
220051,TRAIN_367434,CN,QQW1,Bulk,55.408765,2022-06-16 14:27,D236761,30.0,16,26500,...,0.96,28.2,2.651752,22,108.43,114.13,109.56,2067.433444,0.000595,83.960833
220052,TRAIN_367436,CN,YRT6,Bulk,59.018184,2017-11-11 22:23,J661243,40.0,13,93200,...,NaN,NaN,NaN,6,61.25,62.21,55.70,1333.609109,0.000360,65.850000
220053,TRAIN_367438,SG,GIW5,Container,1.768630,2022-07-14 7:58,Q635545,30.0,6,25000,...,3.36,31.7,2.557156,15,97.73,99.10,95.78,1601.291086,0.002615,0.997500


In [13]:
# datetime 컬럼 처리
train['ATA'] = pd.to_datetime(train['ATA'])
test['ATA'] = pd.to_datetime(test['ATA'])

# datetime을 여러 파생 변수로 변환
for df in [train, test]:
    df['year'] = df['ATA'].dt.year
    df['month'] = df['ATA'].dt.month
    df['day'] = df['ATA'].dt.day
    df['hour'] = df['ATA'].dt.hour
    df['minute'] = df['ATA'].dt.minute
    df['weekday'] = df['ATA'].dt.weekday

# datetime 컬럼 제거
train.drop(columns='ATA', inplace=True)
test.drop(columns='ATA', inplace=True)

In [14]:
train.drop(['SAMPLE_ID'],axis=1,inplace=True)
test.drop(['SAMPLE_ID'],axis=1,inplace=True)

In [15]:
train_x = train.drop(['CI_HOUR'],axis=1)
train_y = train[['CI_HOUR']]

# AG

In [7]:
from autogluon.tabular import TabularDataset, TabularPredictor

train_data = TabularDataset(train)
test_data = TabularDataset(test)


predictor = TabularPredictor(label='CI_HOUR',  eval_metric='mean_absolute_error').fit(train, presets='medium_quality',  ag_args_fit={'num_gpus': 0})
predictor.leaderboard(train, silent=True)


y_pred = predictor.predict(test_data)
y_pred = pd.DataFrame(y_pred, columns=['CI_HOUR'])
submission['CI_HOUR'] = y_pred['CI_HOUR']
predictor.leaderboard(train, silent=True)

No path specified. Models will be saved in: "AutogluonModels\ag-20231005_024957\"
Presets specified: ['medium_quality']
	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20231005_024957\"
AutoGluon Version:  0.8.2
Python Version:     3.8.16
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
Disk Space Avail:   33.32 GB / 511.09 GB (6.5%)
Train Data Rows:    367441
Train Data Columns: 30
Label Column: CI_HOUR
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (2159.130556, 0.0, 61.87712, 170.57522)
	If 'regression' is not the correct problem_type, please manually specify th

[1000]	valid_set's l1: 56.1213
[2000]	valid_set's l1: 54.9496
[3000]	valid_set's l1: 53.8918
[4000]	valid_set's l1: 52.7832
[5000]	valid_set's l1: 52.0745
[6000]	valid_set's l1: 51.5883


KeyboardInterrupt: 

In [ ]:
predictor.leaderboard(train, silent=True)

In [ ]:
submission.to_csv('base.csv',index=False)
submission

# Mljar

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from supervised.automl import AutoML
from sklearn.metrics import mean_squared_error


'''
automl = AutoML(
    algorithms=["CatBoost", "Xgboost", "LightGBM"],
    model_time_limit=30*60,
    start_random_models=10,
    hill_climbing_steps=3,
    top_models_to_improve=3,
    golden_features=True,
    features_selection=False,
    stack_models=True,
    train_ensemble=True,
    explain_level=0,
    validation_strategy={
        "validation_type": "kfold",
        "k_folds": 4,
        "shuffle": False,
        "stratify": True,
    }
)
'''
automl = AutoML(mode = "Compete",
                algorithms = ['LightGBM', 'Xgboost', 'CatBoost'], golden_features=True,
                ml_task = "regression", eval_metric=mean_absolute_error, random_state = 42, total_time_limit=100)

automl.fit(train_x, train_y)

predictions = automl.predict(test)
predictions

AutoML directory: AutoML_7
The task is regression with evaluation metric user_defined_metric
AutoML will use algorithms: ['LightGBM', 'Xgboost', 'CatBoost']
AutoML will stack models
AutoML will ensemble available models
AutoML steps: ['adjust_validation', 'simple_algorithms', 'default_algorithms', 'not_so_random', 'mix_encoding', 'golden_features', 'kmeans_features', 'insert_random_feature', 'features_selection', 'hill_climbing_1', 'hill_climbing_2', 'boost_on_errors', 'ensemble', 'stack', 'ensemble_stacked']
* Step adjust_validation will try to check up to 1 model
1_DecisionTree user_defined_metric 62.792915 trained in 16.24 seconds
Disable stacking for split validation
Skip simple_algorithms because no parameters were generated.
* Step default_algorithms will try to check up to 3 models
There was an error during 2_Default_LightGBM training.
Please check AutoML_7\errors.md for details.
* Step not_so_random will try to check up to 27 models
There was an error during 11_LightGBM trainin

array([7.71755289e+01, 7.71755289e+01, 5.54033668e-03, ...,
       5.54033668e-03, 5.54033668e-03, 7.71755289e+01])

# H2o

# pycaret

In [ ]:
from pycaret.regression import *
import category_encoders

reg = setup(data=train, target='CI_HOUR', use_gpu=True, session_id=313, encoding_method=category_encoders.target_encoder.TargetEncoder(smoothing=10))
reg.pipeline
best = compare_models(n_select=4)
best[2] = tune_model(best[2], optimize='MAE', n_iter=100) #lgbm
best[3] = tune_model(best[3], optimize='MAE', n_iter=100) #xgb
stack_lr = stack_models(best, optimize='MAE', choose_better=True)
stack_finalized = finalize_model(stack_lr)
predictions['CI_HOUR'] = stack_finalized.predict(test)
predictions

# XGB

In [16]:
from xgboost import XGBRegressor
#import xgboost as xgb
import optuna
from optuna import Trial
from optuna.samplers import TPESampler
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import numpy as np


def objective(trial: Trial, X_train, y_train, X_val, y_val):
    params = {
            'iterations':trial.suggest_int("iterations", 300, 1000),
            'learning_rate' : trial.suggest_uniform('learning_rate',0.1, 1),
            'depth': trial.suggest_int('depth',5, 16),
            'min_data_in_leaf': trial.suggest_int('min_data_in_leaf',1,30),
            'reg_lambda': trial.suggest_uniform('reg_lambda',30,100),
            'subsample': trial.suggest_uniform('subsample',0.3,1),
            'random_strength': trial.suggest_uniform('random_strength',10,100),
            'od_wait':trial.suggest_int('od_wait', 10, 150),
            'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations',1,20),
            'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 1, 100),
            "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0., 1.0),
            'random_state' : 9555,
            'verbose' : 0,
        }
    #'task_type' : 'GPU',
    #"eval_metric":'RMSE',
    cat = CatBoostRegressor(**params)
    cat.fit(X_train, y_train, eval_set=[(X_train,y_train),(X_val,y_val)],cat_features=cat_features,
              verbose=False)
    cat_pred = cat.predict(X_val)
    score = mean_absolute_error(y_val, cat_pred)
    return score


In [17]:
from sklearn.preprocessing import LabelEncoder
import bisect
'''
# Categorical 컬럼 인코딩
categorical_features = ['ARI_CO', 'ARI_PO', 'SHIP_TYPE_CATEGORY', 'ID', 'SHIPMANAGER', 'FLAG']
encoders = {}

for feature in tqdm(categorical_features, desc="Encoding features"):
    le = LabelEncoder()
    train[feature] = le.fit_transform(train[feature].astype(str))
    le_classes_set = set(le.classes_)
    test[feature] = test[feature].map(lambda s: '-1' if s not in le_classes_set else s)
    le_classes = le.classes_.tolist()
    bisect.insort_left(le_classes, '-1')
    le.classes_ = np.array(le_classes)
    test[feature] = le.transform(test[feature].astype(str))
    encoders[feature] = le
cat_features = ['ARI_CO', 'ARI_PO', 'SHIP_TYPE_CATEGORY', 'ID', 'SHIPMANAGER', 'FLAG']

# 결측치 처리
train_x.fillna(train_x.mean(), inplace=True)
test.fillna(train_x.mean(), inplace=True)
'''
cat_features = ['ARI_CO', 'ARI_PO', 'SHIP_TYPE_CATEGORY', 'ID', 'SHIPMANAGER', 'FLAG']
# 수치형 변수만 대상으로 결측치 대체
numeric_cols = train_x.select_dtypes(include=[np.number]).columns

# 훈련 데이터의 평균 계산
mean_values = train_x[numeric_cols].mean()

# 결측치 대체
train_x[numeric_cols] = train_x[numeric_cols].fillna(mean_values)
test[numeric_cols] = test[numeric_cols].fillna(mean_values)

In [18]:
train_x = train.drop(['CI_HOUR'],axis=1)
train_y = train[['CI_HOUR']]

kf = KFold(n_splits=5, shuffle=True, random_state=3553)
fold_results = []
test_predictions = []

# 각 Fold에 대해서 Optuna로 하이퍼파라미터 튜닝
for fold, (train_index, val_index) in enumerate(kf.split(train_x)):
    X_train, X_val = train_x.iloc[train_index, :], train_x.iloc[val_index, :]
    y_train, y_val = train_y.iloc[train_index, :], train_y.iloc[val_index, :]

    print(f"Optimizing hyperparameters for fold {fold+1}...")
    study = optuna.create_study(direction='minimize', sampler=TPESampler(seed=1234))
    study.optimize(lambda trial: objective(trial, X_train, y_train, X_val, y_val.values), n_trials=20)

    best_params = study.best_params
    best_score = study.best_value
    
    print(f"Best RMSE for fold {fold+1}: {best_score}")
    print(f"Best hyperparameters for fold {fold+1}: {best_params}")
    
    fold_results.append({
        'fold': fold+1,
        'best_score': best_score,
        'best_params': best_params
    })
    
    # 최적의 하이퍼파라미터로 테스트 데이터 예측
    model = CatBoostRegressor(**best_params)
    model.fit(X_train, y_train, cat_features=cat_features)
    test_pred = model.predict(test)
    test_predictions.append(test_pred)

# 평균 앙상블
final_prediction = np.mean(test_predictions, axis=0)

# 결과 출력
for result in fold_results:
    print(f"Fold {result['fold']}, Best MAE: {result['best_score']}, Best hyperparameters: {result['best_params']}")

# 최종 예측 저장
final_prediction_df = pd.DataFrame(final_prediction, columns=['MLM'])
submission['CI_HOUR'] = final_prediction

[I 2023-10-05 22:57:32,559] A new study created in memory with name: no-name-9bac196a-a60b-465e-951f-a59786c40290


Optimizing hyperparameters for fold 1...


[I 2023-10-05 23:00:06,429] Trial 0 finished with value: 61.04548939421835 and parameters: {'iterations': 434, 'learning_rate': 0.6598978939358486, 'depth': 10, 'min_data_in_leaf': 24, 'reg_lambda': 84.59830656831625, 'subsample': 0.4908148236978491, 'random_strength': 34.8817829628787, 'od_wait': 123, 'leaf_estimation_iterations': 20, 'bagging_temperature': 56.47617424572882, 'colsample_bylevel': 0.35781726995786667}. Best is trial 0 with value: 61.04548939421835.
[I 2023-10-05 23:05:21,120] Trial 1 finished with value: 68.16594682071415 and parameters: {'iterations': 651, 'learning_rate': 0.7151166416549226, 'depth': 13, 'min_data_in_leaf': 12, 'reg_lambda': 69.28373302459374, 'subsample': 0.6521582157154668, 'random_strength': 11.2391604631614, 'od_wait': 118, 'leaf_estimation_iterations': 18, 'bagging_temperature': 5.367498945698209, 'colsample_bylevel': 0.6153961784334937}. Best is trial 0 with value: 61.04548939421835.
[W 2023-10-05 23:10:07,354] Trial 2 failed with parameters: {

KeyboardInterrupt: 

In [18]:
train_x = train.drop(['CI_HOUR'],axis=1)
train_y = train[['CI_HOUR']]

kf = KFold(n_splits=5, shuffle=True, random_state=3553)
fold_results = []
test_predictions = []

# 각 Fold에 대해서 Optuna로 하이퍼파라미터 튜닝
for fold, (train_index, val_index) in enumerate(kf.split(train_x)):
    X_train, X_val = train_x.iloc[train_index, :], train_x.iloc[val_index, :]
    y_train, y_val = train_y.iloc[train_index, :], train_y.iloc[val_index, :]

    print(f"Optimizing hyperparameters for fold {fold+1}...")
    study = optuna.create_study(direction='minimize', sampler=TPESampler(seed=1234))
    study.optimize(lambda trial: objective(trial, X_train, y_train, X_val, y_val.values), n_trials=20)

    best_params = study.best_params
    best_score = study.best_value
    
    print(f"Best RMSE for fold {fold+1}: {best_score}")
    print(f"Best hyperparameters for fold {fold+1}: {best_params}")
    
    fold_results.append({
        'fold': fold+1,
        'best_score': best_score,
        'best_params': best_params
    })
    
    # 최적의 하이퍼파라미터로 테스트 데이터 예측
    model = CatBoostRegressor(**best_params)
    model.fit(X_train, y_train, cat_features=cat_features)
    test_pred = model.predict(test)
    test_predictions.append(test_pred)

# 평균 앙상블
final_prediction = np.mean(test_predictions, axis=0)

# 결과 출력
for result in fold_results:
    print(f"Fold {result['fold']}, Best MAE: {result['best_score']}, Best hyperparameters: {result['best_params']}")

# 최종 예측 저장
final_prediction_df = pd.DataFrame(final_prediction, columns=['MLM'])
submission['CI_HOUR'] = final_prediction

[I 2023-10-05 18:34:50,397] A new study created in memory with name: no-name-b26d2697-0fcd-4dee-9e39-39065c14aaac


Optimizing hyperparameters for fold 1...


[I 2023-10-05 18:35:19,781] Trial 0 finished with value: 42.85265927112725 and parameters: {'iterations': 434, 'learning_rate': 0.6598978939358486, 'depth': 10, 'min_data_in_leaf': 24, 'reg_lambda': 84.59830656831625, 'subsample': 0.4908148236978491, 'random_strength': 34.8817829628787, 'od_wait': 123, 'leaf_estimation_iterations': 20, 'bagging_temperature': 56.47617424572882, 'colsample_bylevel': 0.35781726995786667}. Best is trial 0 with value: 42.85265927112725.
[I 2023-10-05 18:36:53,140] Trial 1 finished with value: 48.80106549563911 and parameters: {'iterations': 651, 'learning_rate': 0.7151166416549226, 'depth': 13, 'min_data_in_leaf': 12, 'reg_lambda': 69.28373302459374, 'subsample': 0.6521582157154668, 'random_strength': 11.2391604631614, 'od_wait': 118, 'leaf_estimation_iterations': 18, 'bagging_temperature': 5.367498945698209, 'colsample_bylevel': 0.6153961784334937}. Best is trial 0 with value: 42.85265927112725.
[I 2023-10-05 18:52:12,454] Trial 2 finished with value: 52.9

KeyboardInterrupt: 

In [ ]:
submission.to_csv('XGB_Optuna_915.csv', index = False, encoding = "utf-8-sig")

In [58]:
train

,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,nAcid,nBase,SpAbs_A,...,MACCS_key_158,MACCS_key_159,MACCS_key_160,MACCS_key_161,MACCS_key_162,MACCS_key_163,MACCS_key_164,MACCS_key_165,MLM,HLM
0,3.259,400.495,5,2,8,3.259,117.37,0,0,35.689316,...,1,1,1,1,1,1,1,1,26.010,50.680
1,2.169,301.407,2,1,2,2.172,73.47,0,0,26.575899,...,1,0,1,1,1,1,1,1,29.270,50.590
2,1.593,297.358,5,0,3,1.585,62.45,2,1,29.802128,...,1,0,1,1,1,1,0,1,5.586,80.892
3,4.771,494.652,6,0,5,3.475,92.60,0,1,45.884166,...,1,1,1,1,1,1,1,1,5.710,2.000
4,2.335,268.310,3,0,1,2.337,42.43,0,0,26.308663,...,1,1,1,1,1,1,1,1,93.270,99.990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3466,3.409,396.195,3,1,5,3.409,64.74,0,0,30.902711,...,1,0,1,1,1,0,1,1,1.556,3.079
3467,1.912,359.381,4,1,3,1.844,77.37,0,0,35.887372,...,1,1,1,1,1,1,1,1,35.560,47.630
3468,1.941,261.320,3,1,6,2.124,70.14,0,0,23.546531,...,1,1,1,1,1,1,1,1,56.150,1.790
3469,0.989,284.696,5,1,5,0.989,91.51,0,0,23.936088,...,1,1,0,1,1,1,1,1,0.030,2.770
